In [6]:
# Author: Arthur Prigent
#EMail: aprigent@ictp.it

In [6]:
# Author: Arthur Prigent
# Email: aprigent@ictp.it

#### DATA ####
# https://downloads.psl.noaa.gov/Datasets/
from functions import *
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image

2024-03-26 10:21:36.591004


In [7]:
path_ersstv5 = 'https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
(ssta_atl3_norm_ersst,ssta_aba_norm_ersst,
 ssta_nino34_norm_ersst,ssta_dni_norm_ersst,
 ssta_cni_norm_ersst,ssta_nni_norm_ersst,iod_ersst) = read_data_compute_anomalies_ersstv5(path_ersstv5)

In [42]:
last_timesteps = [str(ssta_atl3_norm_ersst.time[-3].values)[:7],
str(ssta_atl3_norm_ersst.time[-2].values)[:7],
str(ssta_atl3_norm_ersst.time[-1].values)[:7]]



['2023-12', '2024-01', '2024-02']

In [44]:
import os
mail_sender = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")



In [45]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Set your email and password
sender_email = mail_sender
sender_password = password

# Set the recipient email address
recipient_email = "aprigent@ictp.it"
smtp_server = "smtp.gmail.com"
smtp_port = 587

# Start the server
server = smtplib.SMTP(smtp_server, smtp_port)
server.starttls()

# Log in to your Gmail account
server.login(sender_email, sender_password)

# Create the email message
subject = "Tropical Atlantic Variability update"
body = ("Hello!\n"
        f"Please find at the following link a little update on the current state of the tropics for the last three timesteps:\n{last_timesteps} \n"
        "Last three standardized anomalies from ERSSTv5:\n"
        f"ATL3: {np.round(ssta_atl3_norm_ersst[-3:],2).values}. \n"
        f"ABA: {np.round(ssta_aba_norm_ersst[-3:],2).values}. \n"
        f"NINO3.4: {np.round(ssta_nino34_norm_ersst[-3:],2).values}. \n"
        f"DNI: {np.round(ssta_dni_norm_ersst[-3:],2).values}. \n"

        "Best regards,\n"
        "Arthur Prigent\n\n"
        "https://aprig.github.io/ATL3_ABA_SSTa/automatic_time_series_SST.html")

message = MIMEMultipart()
message['From'] = sender_email
message['To'] = recipient_email
message['Subject'] = subject
message.attach(MIMEText(body, 'plain'))

# Send the email
server.sendmail(sender_email, recipient_email, message.as_string())

# Quit the server
server.quit()


(221,
 b'2.0.0 closing connection bu16-20020a056000079000b00341cd8e04a0sm4902279wrb.105 - gsmtp')